# VN-Quant Stockformer Training
**Train 102 Vietnamese stock prediction models on Google Colab**

## Setup
1. Runtime > Change runtime type > GPU (T4/V100/A100)
2. Mount Google Drive
3. Upload data from host server
4. Run training
5. Download models back to host

In [ ]:
# 1. Check GPU
!nvidia-smi
import torch
print(f"\nPyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# 2. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create project folder
!mkdir -p /content/drive/MyDrive/VNQuant/data
!mkdir -p /content/drive/MyDrive/VNQuant/models
!mkdir -p /content/drive/MyDrive/VNQuant/checkpoints

In [ ]:
# 3. Install dependencies
!pip install pandas numpy torch scikit-learn pyarrow tqdm -q

In [ ]:
# 4. Define Stockformer Model
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=100):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

class StockformerSimple(nn.Module):
    def __init__(self, input_dim=5, d_model=64, nhead=4, num_layers=2, 
                 dim_feedforward=128, dropout=0.1, forecast_days=5):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward,
            dropout=dropout, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output_proj = nn.Linear(d_model, forecast_days)
        self.forecast_days = forecast_days

    def forward(self, x):
        x = self.input_proj(x)
        x = self.pos_encoder(x)
        x = self.transformer(x)
        x = x[:, -1, :]
        return self.output_proj(x)

print("Model defined!")

In [ ]:
# 5. Training Functions
def prepare_data(df, seq_len=20):
    """Prepare sequences for training"""
    features = ['open', 'high', 'low', 'close', 'volume']
    data = df[features].values
    
    # Normalize
    mean = data.mean(axis=0)
    std = data.std(axis=0) + 1e-8
    data_norm = (data - mean) / std
    
    X, y = [], []
    for i in range(len(data_norm) - seq_len - 5):
        X.append(data_norm[i:i+seq_len])
        # Target: 5-day returns
        future_close = data[i+seq_len:i+seq_len+5, 3]  # close prices
        current_close = data[i+seq_len-1, 3]
        returns = (future_close - current_close) / current_close * 100
        y.append(returns)
    
    return np.array(X), np.array(y), {'mean': mean, 'std': std}

def train_model(symbol, data_path, save_path, epochs=100, batch_size=32):
    """Train model for one symbol"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Load data
    df = pd.read_parquet(data_path)
    if len(df) < 50:
        return {'symbol': symbol, 'success': False, 'error': 'Insufficient data'}
    
    # Prepare
    X, y, norm_params = prepare_data(df)
    if len(X) < 20:
        return {'symbol': symbol, 'success': False, 'error': 'Insufficient sequences'}
    
    # Split
    split = int(len(X) * 0.8)
    X_train, X_val = X[:split], X[split:]
    y_train, y_val = y[:split], y[split:]
    
    # Tensors
    X_train = torch.FloatTensor(X_train).to(device)
    y_train = torch.FloatTensor(y_train).to(device)
    X_val = torch.FloatTensor(X_val).to(device)
    y_val = torch.FloatTensor(y_val).to(device)
    
    # Model
    model = StockformerSimple().to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
    
    best_val_loss = float('inf')
    
    for epoch in range(epochs):
        model.train()
        # Mini-batch training
        perm = torch.randperm(len(X_train))
        total_loss = 0
        for i in range(0, len(X_train), batch_size):
            idx = perm[i:i+batch_size]
            optimizer.zero_grad()
            pred = model(X_train[idx])
            loss = criterion(pred, y_train[idx])
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        scheduler.step()
        
        # Validation
        model.eval()
        with torch.no_grad():
            val_pred = model(X_val)
            val_loss = criterion(val_pred, y_val).item()
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save({
                'model_state_dict': model.state_dict(),
                'norm_params': norm_params,
                'val_loss': val_loss
            }, save_path)
    
    return {
        'symbol': symbol,
        'success': True,
        'val_loss': best_val_loss,
        'samples': len(X)
    }

print("Training functions ready!")

In [ ]:
# 6. List available stock data
DATA_DIR = Path('/content/drive/MyDrive/VNQuant/data')
MODEL_DIR = Path('/content/drive/MyDrive/VNQuant/models')

parquet_files = list(DATA_DIR.glob('*.parquet'))
symbols = [f.stem for f in parquet_files]

print(f"Found {len(symbols)} stocks to train")
print(f"Symbols: {symbols[:20]}..." if len(symbols) > 20 else f"Symbols: {symbols}")

In [ ]:
# 7. TRAIN ALL MODELS
from datetime import datetime
import json

print("="*60)
print(f"Starting training at {datetime.now()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print(f"Stocks: {len(symbols)}")
print("="*60)

results = []
start_time = datetime.now()

for i, symbol in enumerate(tqdm(symbols, desc="Training")):
    data_path = DATA_DIR / f"{symbol}.parquet"
    save_path = MODEL_DIR / f"{symbol}_stockformer_simple_best.pt"
    
    result = train_model(symbol, data_path, save_path, epochs=100, batch_size=64)
    results.append(result)
    
    if (i + 1) % 10 == 0:
        success = len([r for r in results if r['success']])
        print(f"\nProgress: {i+1}/{len(symbols)} | Success: {success}")

# Summary
duration = (datetime.now() - start_time).total_seconds() / 3600
successful = [r for r in results if r['success']]
failed = [r for r in results if not r['success']]

print("\n" + "="*60)
print("TRAINING COMPLETE")
print("="*60)
print(f"Duration: {duration:.1f} hours")
print(f"Success: {len(successful)}/{len(symbols)}")
print(f"Failed: {len(failed)}")

if failed:
    print(f"Failed symbols: {[r['symbol'] for r in failed]}")

# Save results
with open(MODEL_DIR / 'training_results.json', 'w') as f:
    json.dump({
        'date': datetime.now().isoformat(),
        'duration_hours': duration,
        'total': len(symbols),
        'success': len(successful),
        'failed': len(failed),
        'results': results
    }, f, indent=2)

In [ ]:
# 8. Verify trained models
trained_models = list(MODEL_DIR.glob('*_stockformer_simple_best.pt'))
print(f"\nTrained models saved: {len(trained_models)}")
print(f"Location: {MODEL_DIR}")
print("\nDownload these models to your host server's models/ folder")